In [1]:
# Import libraries
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, LeakyReLU, BatchNormalization, GaussianNoise
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.optimizers import SGD
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
from keras.regularizers import l1

# Preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler, Normalizer
from sklearn.impute import SimpleImputer

# Evaluating
from sklearn.model_selection import GridSearchCV, train_test_split
import matplotlib.pyplot as plt

# Saving model
from keras.models import load_model

imputer = SimpleImputer()
MMS = MinMaxScaler()
RS = RobustScaler()
SS = StandardScaler()
Norm = Normalizer()

In [3]:
# Get data and merge together

data_current = pd.read_csv('./Scraping/Current Stats and Games.csv', parse_dates=['Date'])
data_prev = pd.read_csv('./Scraping/Previous Stats and Games.csv', parse_dates=['Date'])

print('Current NA values: ',data_current.isna().sum().sum())
print('Prev Year NA values: ',data_prev.isna().sum().sum())

Current NA values:  52828
Prev Year NA values:  42270


In [4]:
# Home win = 49% baseline
# Fav win = 46.6% baseline

y = data_current['Home Win']
x = data_current.loc[:,'H  #Bat':]
y = data_prev['Home Win']
x = data_prev.loc[:,'H  #Bat':]
x_with_odds = x.copy()
x_with_odds[['Home Odds', 'Vis Odds']] = data_current[['Home Odds', 'Vis Odds']]
# x_dates = x.copy()

In [5]:
# Split columns with hyphens
# Create function to use on both current and prev stats

x = x.astype(str)

cols_to_delim = []

for col in x.columns:
    result = x[col].str.contains(pat='\d-\d')
    if result.any():
        cols_to_delim.append(col)

for col in cols_to_delim:
        x[[col + '1', col + '2']] = x[col].str.split('-', expand=True)
        del x[col]
        
x = x.astype(float)


# Impute Scale/Normalise the data

x_impute = imputer.fit_transform(x, y)

x_scale = MMS.fit_transform(x_impute)
# x_scale

In [18]:
# Create the model

# BATCH NORMALIZATION
# batch size = 1, batch_size = len(x_train) batch_size = len(x_train)//16
# batch_size = 500 (53.2%), batch_size = 1 (54%), batch_size = len(x_train) (51%), batch_size = len(x_train)//16 (52.68%)
# BatchNorm Layers are terrible (50%)

# WEIGHT REGULARIZATION
# kernel_regularizer=l2(0.01) Took too long to run
# LEARNING RATE SCHEDULE / ReduceLRonplateau (54%)
# Activity regularization

# Adding Noise
# GaussianNoise (52.68%)
# PRE-TRAINING ON LAST YEAR DATA

model = Sequential()
model.add(Dense(128, input_dim=x.shape[1], activation='relu'))
model.add(Dense(236, activation='relu'))
model.add(Dense(118, activation='relu'))
model.add(Dense(59, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(15, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

sgd = SGD(lr=0.0001, momentum=0)
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss='squared_hinge', optimizer=sgd, metrics=['accuracy'])

# LEARNING RATE SCHEDULE
rlrp = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=5, min_delta=1E-7, verbose=1)
# Early Stopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
model.fit(x_scale, y, epochs=20, batch_size=100, validation_split=0.2, verbose=1, callbacks=[rlrp])

# model.fit(x_scale, y, epochs=10, batch_size=1, validation_split=0.2, verbose=1)

Epoch 1/20
773/773 [==============================] - 3s 4ms/step - loss: 1.1661 - accuracy: 0.4650 - val_loss: 1.1783 - val_accuracy: 0.4733
Epoch 2/20
773/773 [==============================] - 3s 3ms/step - loss: 1.1735 - accuracy: 0.4719 - val_loss: 1.1707 - val_accuracy: 0.4733
Epoch 3/20
773/773 [==============================] - 3s 3ms/step - loss: 1.1708 - accuracy: 0.4745 - val_loss: 1.1635 - val_accuracy: 0.4733
Epoch 4/20
773/773 [==============================] - 3s 3ms/step - loss: 1.1533 - accuracy: 0.4690 - val_loss: 1.1563 - val_accuracy: 0.4733
Epoch 5/20
773/773 [==============================] - 3s 3ms/step - loss: 1.1460 - accuracy: 0.4689 - val_loss: 1.1490 - val_accuracy: 0.4733
Epoch 6/20
773/773 [==============================] - 2s 3ms/step - loss: 1.1439 - accuracy: 0.4719 - val_loss: 1.1415 - val_accuracy: 0.4733

Epoch 00006: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
Epoch 7/20
773/773 [==============================] - 2s 3ms/step -

In [9]:
# # Checkpointing models
# checkpoint = ModelCheckpoint('Weights.hdf5', monitor='accuracy', save_best_only=True, mode='max')
# callbacks_list = [checkpoint]
# model.fit(x, y, epochs=5, verbose=0, callbacks=callbacks_list)

In [106]:
# Grid Search with Keras

def create_model(learn_rate=1e-6, momentum=0.0, init_mode='uniform', activation='relu', dropout_rate=0.0, weight_constraint=0):
    model = Sequential()
    model.add(Dense(128, input_dim=x.shape[1], activation=activation, kernel_initializer=init_mode, kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(236, activation=activation, kernel_initializer=init_mode, kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(118, activation=activation, kernel_initializer=init_mode, kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(59, activation=activation, kernel_initializer=init_mode, kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(30, activation=activation, kernel_initializer=init_mode, kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(15, activation=activation, kernel_initializer=init_mode, kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(4, activation=activation, kernel_initializer=init_mode, kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid', kernel_initializer=init_mode))
    
    optimizer = SGD(lr=learn_rate, momentum=momentum)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

estimator = KerasClassifier(build_fn=create_model, epochs=10)

# Grid Search - Use a coarse grid search first then zoom in on opportunities

param_grid = dict(
    epochs=[10, 20, 30, 40],
    batch_size=[50, 100, 150],
    optimizer=['SGD', 'Adam', 'RMSProp', 'Adagrad', 'Adadelta', 'Adamax', 'Nadam'],
    learn_rate=[1e-6, 1e-7, 1e-8],
    momentum=[0.0, 0.2,],
    init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform'],
    activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear'],
    weight_constraint = [0, 1, 2, 3, 4, 5, 6],
    dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
)

grid = GridSearchCV(estimator=estimator, param_grid=param_grid, n_jobs=-1, cv=10)
grid_result = grid.fit(x_scale, y)
print('Best score: ',grid_result.best_score_)
print('Best params: ', grid_result.best_params_)

Epoch 1/10
3019/3019 [==============================] - 9s 3ms/step - loss: 0.6931 - accuracy: 0.5296
Epoch 2/10
3019/3019 [==============================] - 9s 3ms/step - loss: 0.6931 - accuracy: 0.5296
Epoch 3/10
3019/3019 [==============================] - 8s 3ms/step - loss: 0.6931 - accuracy: 0.5296
Epoch 4/10
3019/3019 [==============================] - 9s 3ms/step - loss: 0.6931 - accuracy: 0.5296
Epoch 5/10
3019/3019 [==============================] - 9s 3ms/step - loss: 0.6931 - accuracy: 0.5296
Epoch 6/10
3019/3019 [==============================] - 11s 4ms/step - loss: 0.6931 - accuracy: 0.5296
Epoch 7/10
3019/3019 [==============================] - 11s 4ms/step - loss: 0.6931 - accuracy: 0.5296
Epoch 8/10
3019/3019 [==============================] - 10s 3ms/step - loss: 0.6931 - accuracy: 0.5296
Epoch 9/10
3019/3019 [==============================] - 32s 11ms/step - loss: 0.6931 - accuracy: 0.52960s - loss: 0.6
Epoch 10/10
3019/3019 [==============================] - 55s 18

In [107]:
# acc = 0.5296205997467041
# loss = 0.6931

# Plot Model
# history = model.fit()
# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('Model Accuracy')
# plt.ylabel('Accuracy')
# plt.xlabel('Epochs')
# plt.legend(['Train', 'Test'], loc='upper_left')
# plt.show()

In [7]:
# Saving Model
# model.save('New Model.h5')

# Load Model
model = load_model('Model.h5')

model.fit(x_scale, y, epochs=10, batch_size=500, validation_split=0.2, verbose=1)

Epoch 1/10
155/155 [==============================] - 1s 4ms/step - loss: 0.6872 - accuracy: 0.5439 - val_loss: 0.6877 - val_accuracy: 0.5402
Epoch 2/10
155/155 [==============================] - 1s 3ms/step - loss: 0.6872 - accuracy: 0.5437 - val_loss: 0.6877 - val_accuracy: 0.5401
Epoch 3/10
155/155 [==============================] - 1s 4ms/step - loss: 0.6872 - accuracy: 0.5438 - val_loss: 0.6877 - val_accuracy: 0.5402
Epoch 4/10
155/155 [==============================] - 1s 3ms/step - loss: 0.6872 - accuracy: 0.5440 - val_loss: 0.6877 - val_accuracy: 0.5401
Epoch 5/10
155/155 [==============================] - 1s 4ms/step - loss: 0.6872 - accuracy: 0.5438 - val_loss: 0.6877 - val_accuracy: 0.5401
Epoch 6/10
155/155 [==============================] - 1s 4ms/step - loss: 0.6872 - accuracy: 0.5439 - val_loss: 0.6877 - val_accuracy: 0.5401
Epoch 7/10
155/155 [==============================] - 1s 4ms/step - loss: 0.6872 - accuracy: 0.5437 - val_loss: 0.6877 - val_accuracy: 0.5401
Epoch 

Model: "sequential_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_264 (Dense)            (None, 128)               30336     
_________________________________________________________________
dense_265 (Dense)            (None, 236)               30444     
_________________________________________________________________
dense_266 (Dense)            (None, 118)               27966     
_________________________________________________________________
dense_267 (Dense)            (None, 59)                7021      
_________________________________________________________________
dense_268 (Dense)            (None, 30)                1800      
_________________________________________________________________
dense_269 (Dense)            (None, 15)                465       
_________________________________________________________________
dense_270 (Dense)            (None, 4)               

In [ ]:
#### USE TRAIN TEST VALIDATE
train and valid in model.fit(validation=validation split)
model.evaluate(test split)

In [ ]:
## PRE-TRAINING 53.04% Test Acc


# batch size = 1, batch_size = len(x_train) batch_size = len(x_train)//16
# ReduceLRonplateau
# rlrp = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_delta=1E-7, verbose=1)
# model.fit(trainX, trainy, validation_data=(testX, testy), epochs=300, verbose=0, callbacks=[rlrp])

# Create train and test
train_percent = 0.15
n_train = int(len(x_scale) * train_percent)
x_train, x_test = x_scale[:n_train, :], x_scale[n_train:, :]
y_train, y_test = y[:n_train], y[n_train:]

epochs = 20
batch_size = 1

# Base model
def get_base_model(x_train, y_train):
    model = Sequential()
    model.add(Dense(236, input_dim=236, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    
    opt = SGD(lr=0.0001, momentum=0.9)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)
    return model

# Eval model
def evaluate_model(model, x_train, y_train, x_test, y_test):
    _, train_acc = model.evaluate(x_train, y_train, verbose=0)
    _, test_acc = model.evaluate(x_test, y_test, verbose=0)
    return train_acc, test_acc

# Add one layer only and set trainable=False
def add_layer(model, x_train, y_train):
    output_layer = model.layers[-1]
    model.pop()
    for layer in model.layers:
        layer.trainable = False
    model.add(Dense(236, activation='relu'))
    model.add(Dropout(0.1))
    model.add(output_layer)
    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)
    
model = get_base_model(x_train, y_train)

scores = dict()
train_acc, test_acc = evaluate_model(model, x_train, y_train, x_test, y_test)
print('> layers=%d, train=%.3f, test=%.3f' % (len(model.layers), train_acc, test_acc))
scores[len(model.layers)] = (train_acc, test_acc)

n_layers = 9

for i in range(n_layers):
    add_layer(model, x_train, y_train)
    train_acc, test_acc = evaluate_model(model, x_train, y_train, x_test, y_test)
    print('> layers=%d, train=%.3f, test=%.3f' % (len(model.layers), train_acc, test_acc))
    scores[len(model.layers)] = (train_acc, test_acc)
    
plt.plot(list(scores.keys()), [scores[k][0] for k in scores.keys()], label='train', marker='.')
plt.plot(list(scores.keys()), [scores[k][1] for k in scores.keys()], label='test', marker='.')
plt.legend()
plt.show()

In [82]:
# go through all of MLM and DeepLizard

# model.save('Model 58% Train Acc.h5') # Over fitting train = 60%, test = 53.4%
# Model.h5 = 54% train acc, 54% test acc

# Pretraining | Increase in 3% in train, no increase in test
# Add in the NBA features - Check | slight decrease
# Activity regularization | Decrease in 2%
# Batchnorm | Increase in training (60%) no increase in testing (53.4%)
# Glorot_uniform/glorot_normal = batch norming

# Lesson 3 Learning Rate Schedule
# Add in power rankings
# Add in ELO ratings
# Use LSTM layers
# Use predictions of > 60%, 70%, 80%, 90%, 95%

In [ ]:
# Calculate for each year
    # Home wins
    # Fav wins
    # Total Fav Wins

home_fav = data_current[data_current['Home Odds'] < data_current['Vis Odds']]
vis_fav = data_current[data_current['Home Odds'] > data_current['Vis Odds']]
home_fav_win = home_fav[home_fav['Home Win'] == True]
vis_fav_win = vis_fav[vis_fav['Home Win'] == False]

print('Home Favourite Wins: ',len(home_fav_win) / len(home_fav)) # 57% HOME FAV WINS
print('Visitor Favourite Wins: ',len(vis_fav_win) / len(vis_fav)) # 56% VIS FAV WINS
print('Favourite Wins: ',(len(home_fav_win) + len(vis_fav_win)) / (len(home_fav) + len(vis_fav))) # 56.6% FAVOURITE WINS

print('Home Wins: ',len(data_current[data_current['Home Win'] == True]) / len(data_current)) # 53% HOME WINS

0.5296206568103698